In [ ]:
import heapq
import multiprocessing
import os

import matplotlib.pyplot as plt
from matplotlib import colors

import numpy as np
import torch
import pandas as pd

import pickle

import droplet_approximation

from scipy.integrate import solve_ivp

# New Graphing Overview
This notebook provides examples for working with the new graphing function in `analysis.py`. The four new functions are:
1. `plot_droplet_size_temperatures`           - plots provided radius/temperature np array time series data
2. `plot_droplet_size_temperatures_domain`    - plots BDF/iterative model solution for given droplet parameters
3. `plot_droplet_size_temperatures_dataframe` - plots time series data for specified evaluations from a particle dataframe
4. `plot_droplet_size_temperatures_scoring`   - plots time series data and deviations for a particle dataframe/score report

## Setup/Settings

In [ ]:
# Do NOT edit this cell. Instead, make any changes you want in the cell below it by setting
# these variables.

# Commit for each model
current_SHA = "79a3442545133bfe38cecf9b67ab928538842b23"

# Change this to fit wherever testing data is stored. TODO: update this to match particles exploration
simulation_name = "Pi Chamber 1way RH103"
particles_root  = "/groups/drichte2/droplet_approximation/data/simulations/pi_chamber-1way-rh103/particles"
#particles_root  = "../data/particles"
dirs_per_level  = 256

model_path = "/afs/crc.nd.edu/group/RichterLab/droplet_approximation/models/box-narrow_uniform_high_res-mlp_4layer-b=1024-lr=1e-3_halvingschedule-l2reg=1e-6_checkpoint.pt"

score_report_path = "/groups/drichte2/droplet_approximation/data/analysis/tmp/scoring_report-pi_chamber_1way_rh103-be_vs_bdf_iterative.pkl"


In [ ]:
model = droplet_approximation.ResidualNet()
param_list = droplet_approximation.load_model_checkpoint( model_path, model )

In [ ]:
droplet_approximation.set_parameter_ranges(param_list[0])

## Graphing Particles Rolled from Domain

In [ ]:
# Roll background parameters
input_parameters = droplet_approximation.scale_droplet_parameters( np.random.uniform( -1, 1, 6 ) )
input_parameters[3] = input_parameters[1] + 0.5

### Graphing the Numpy Arrays directly

`plot_droplet_size_temperature` allows us to directly graph data from numpy arrays.<br>
By default, it will automatically generate comparison plots if there are two or more radius/temperature time series plotted.

In [ ]:
t_eval             = np.linspace(0.1, 10.0, 100)
droplet_parameters = np.tile( input_parameters, (100, 1) )

bdf_output         = droplet_approximation.do_iterative_bdf( droplet_parameters, t_eval )
model_output       = droplet_approximation.do_iterative_inference( droplet_parameters, t_eval, model, "cpu" )

In [ ]:
# Example with just one time series
size_temperatures = {
    "iterative bdf": bdf_output
}

droplet_approximation.plot_droplet_size_temperatures( t_eval, size_temperatures ) 

In [ ]:
# Example with two
size_temperatures = {
    "iterative bdf": bdf_output,
    "mlp": model_output
}

droplet_approximation.plot_droplet_size_temperatures( t_eval, size_temperatures ) 

### Graph just the BDF solution
To make things easier, there is a function `plot_droplet_size_temperature_domain` that takes in some droplet parameter array rolled from the input domain and integrates it out to `final_time` (defaults to `10` seconds).

In [ ]:
# Graph just the truth value
droplet_approximation.plot_droplet_size_temperatures_domain(input_parameters )

### Graph the BDF solution against the model
If supplied a model, the function will evaluate the model as well, comparing the two solutions in the second row of plots.<br><br>
In model evaluation mode, `dt` can be specified for `do_iterative_inference`. If this is not provided, the function uses the average of the log time scale as the default for `dt`.

In [ ]:
droplet_approximation.plot_droplet_size_temperatures_domain( input_parameters, model, dt=0.10 )

## Graphing Particles from Trace Data

In [ ]:
GRAPH_COUNT = 1

ids_index = np.fromfile( particles_root + "/particles.index", dtype=np.int32 )
target_particle_ids = np.random.choice( ids_index, GRAPH_COUNT )
df = droplet_approximation.read_particles_data( particles_root, target_particle_ids, dirs_per_level, evaluations={"bdf iterative": "bdf_iterative"}, cold_threshold=284.0 )
df

### Graphing Dataframe Evaluation Particles
The interface only requires a row of a particles dataframe and the evaluation tags to plot.<br>
To graph just one evaluation, one can supply just a string instead of a list. The following cell provides examples of both.

In [ ]:
reference_tag = "be"
comparison_tag = "bdf iterative"

for particle_id in target_particle_ids:
    particle_df      = df.loc[particle_id]
    droplet_approximation.plot_droplet_size_temperatures_dataframe( particle_df, reference_tag )
    droplet_approximation.plot_droplet_size_temperatures_dataframe( particle_df, [reference_tag, comparison_tag] )


### Background parameters
All of the new graphing functions accept an optional parameter `background_parameters`.<br>
It is formatted as a dictionary between the label for the variable and its time series data.<br><br>
These could be just about anything (other rows of the dataframe, be failures, CUSUM data, dydt data, etc.)

In [ ]:
for particle_id in target_particle_ids:
    particle_df    = df.loc[particle_id]

    background_parameters = {
        "Relative Humidity (%)": particle_df["relative humidities"],
        "Particle/Air Temperature Delta (K)":  particle_df["input be temperatures"] - particle_df["air temperatures"],
        "Salt Mass Ratio (m_s/m_w)": particle_df["salt masses"] / (1000*4/3*np.pi*particle_df["input be radii"]**3)
    }

    droplet_approximation.plot_droplet_size_temperatures_dataframe( particle_df, [reference_tag, comparison_tag], background_parameters=background_parameters )

### Score Report Graphing
The last graphing function is `plot_droplet_size_temperature_scoring`. It accepts a particle dataframe and a scoring report. It graphs the comparison between the reference and comparison tags stored in the scoring report, adding labels for any deviations encountered.

In [ ]:
with open( score_report_path, "rb" ) as score_file:
    score_report = pickle.load( score_file )

In [ ]:
for particle_id in target_particle_ids:
    droplet_approximation.plot_droplet_size_temperatures_scoring( particle_df, score_report )


### Title String
All new graphing functions also have an optional keywork `title_string` which overrides the default title string for the plot. e.g.:

In [ ]:
for particle_id in target_particle_ids:
    title_string = "Comparison between {:s}/{:s} \n On particle {:d} from trace {:s}\n ppNRMSE: {:.4f} \n SHA: {:s}".format( reference_tag,
                                                                                                                             comparison_tag,
                                                                                                                             particle_id, 
                                                                                                                             simulation_name,
                                                                                                                             score_report.per_particle_nrmse[ particle_id ],
                                                                                                                             current_SHA )
    droplet_approximation.plot_droplet_size_temperatures_scoring( particle_df, score_report, title_string=title_string )